In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape

#### Scale Features to Speed-up Convergence

In [3]:
X = housing.data
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)

print(X_mean.shape)
print(X_std.shape)

(8,)
(8,)


In [4]:
scaled_X = (X - X_mean) / X_std

In [5]:
scaled_X.mean(axis=0)

array([ 6.60969987e-17,  5.50808322e-18,  6.60969987e-17, -1.06030602e-16,
       -1.10161664e-17,  3.44255201e-18, -1.07958431e-15, -8.52651283e-15])

In [6]:
scaled_X.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [7]:
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_X]

#### Use Placeholder & TensorBoard

In [8]:
from datetime import datetime

# create a unique directory for each run
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "%s/05_gradient_descent_california_run_%s"%(root_logdir, now)

In [9]:
learning_rate = 0.01
training_epochs = 1000
cost_history = np.empty(shape=[1], dtype=float)

X = tf.placeholder(tf.float32, [None, n + 1], name="X")
y = tf.placeholder(tf.float32, [None, 1], name="y")
theta = tf.Variable(tf.ones([n + 1, 1]))

y_pred = tf.matmul(X, theta)

with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error))

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

# create a saver node
saver = tf.train.Saver({"weights" : theta})

# for tensor board
mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
init = tf.global_variables_initializer()
feed = {X:scaled_housing_data_plus_bias, y:housing.target.reshape(-1, 1)}

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        if epoch % 100 == 0:
            print("epoch %i, mse=%.5f"%(epoch, sess.run(mse, feed_dict=feed)))
            
            # log tensor board
            summary_str = mse_summary.eval(feed_dict=feed)
            file_writer.add_summary(summary_str, epoch)
            
        sess.run(training_op, feed_dict=feed)
        cost_history = np.append(cost_history, sess.run(mse, feed_dict=feed))
        
    best_theta = theta.eval()
    save_path = saver.save(sess, "outputs/ca_housing.ckpt")

# end tensorboard logging
file_writer.close()

best_theta

epoch 0, mse=7.87810
epoch 100, mse=0.55918
epoch 200, mse=0.52776
epoch 300, mse=0.52476
epoch 400, mse=0.52438
epoch 500, mse=0.52433
epoch 600, mse=0.52432
epoch 700, mse=0.52432
epoch 800, mse=0.52432


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(cost_history)),cost_history)
plt.axis([0, training_epochs, 0, 10])
plt.show()

#### Restore Model

In [ ]:
saver = tf.train.import_meta_graph("outputs/ca_housing.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "outputs/ca_housing.ckpt")